<a href="https://colab.research.google.com/github/SamratRode/Fake-news-detection-using-BERT-and-hybrid-model/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
real_data=pd.read_csv("/content/drive/MyDrive/dataset/Fake_News_Detection/True.csv")
fake_data=pd.read_csv("/content/drive/MyDrive/dataset/Fake_News_Detection/Fake.csv")

In [ ]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
real_data

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
#Removing Date Attribute
real_data = real_data.iloc[:, 0:-1]
fake_data = fake_data.iloc[:, 0:-1]

In [ ]:
real_data

,title,text,subject
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews


In [ ]:
print(real_data.shape)
print(fake_data.shape)

(21417, 3)
(23481, 3)


In [ ]:
fake_rate=(fake_data.shape[0])/((fake_data.shape[0])+real_data.shape[0])*100
print("Percentage of Fake data is: ", fake_rate)

Percentage of Fake data is:  52.29854336496058


In [ ]:
real_rate=100-fake_rate
print("Percentage of Real data is: ", real_rate)

Percentage of Real data is:  47.70145663503942


In [ ]:
#Since the amount of fake news is high, the data samples are biased towards fake news
#The differece between the percentage is not large and hence we can still have fair accuracy while training our model
print("Data is biased but can be worked with...")

Data is biased but can be worked with...


In [ ]:
fake_data['label']=0
real_data['label']=1

In [ ]:
fake_data.isna().sum

<bound method DataFrame.sum of        title   text  subject  label
0      False  False    False  False
1      False  False    False  False
2      False  False    False  False
3      False  False    False  False
4      False  False    False  False
...      ...    ...      ...    ...
23476  False  False    False  False
23477  False  False    False  False
23478  False  False    False  False
23479  False  False    False  False
23480  False  False    False  False

[23481 rows x 4 columns]>

In [ ]:
real_data.isna().sum

<bound method DataFrame.sum of        title   text  subject  label
0      False  False    False  False
1      False  False    False  False
2      False  False    False  False
3      False  False    False  False
4      False  False    False  False
...      ...    ...      ...    ...
21412  False  False    False  False
21413  False  False    False  False
21414  False  False    False  False
21415  False  False    False  False
21416  False  False    False  False

[21417 rows x 4 columns]>

In [ ]:
data = fake_data
data=pd.concat([fake_data,real_data], axis=0, ignore_index=True)
data

,title,text,subject,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,0
...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,1


In [ ]:
data_title_label = data.iloc[:, [0,3]]

In [ ]:
data_title_label

,title,label
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 44898):
    title = re.sub('[^a-zA-Z]', ' ', data_title_label['title'][i]) # review to contain only letters a to z and if the character is removed replace it by so+pace
    title = title.lower() # review to contain only lower case letter
    title = title.split() # Splitting the string into different words
    ps = PorterStemmer() # Creating object of the class
    title = [ps.stem(word) for word in title if not word in set(stopwords.words('english'))]  # Set here is used only for faster execution of code in case there are too many words...
    title = ' '.join(title) # Converting the list back to string
    corpus.append(title)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=15000)
X = cv.fit_transform(corpus).toarray()
y = data_title_label.iloc[:, 1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.25,
                                                   random_state=0)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
1053/1053 [==============================] - 6s 5ms/step - loss: 0.2761 - accuracy: 0.9132
Epoch 2/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0991 - accuracy: 0.9639
Epoch 3/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0700 - accuracy: 0.9757
Epoch 4/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0533 - accuracy: 0.9820
Epoch 5/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0422 - accuracy: 0.9858
Epoch 6/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0336 - accuracy: 0.9892
Epoch 7/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0271 - accuracy: 0.9912
Epoch 8/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0228 - accuracy: 0.9922
Epoch 9/100
1053/1053 [==============================] - 5s 5ms/step - loss: 0.0190 - accuracy: 0.9941
Epoch 10/100
1053/1053 [==============================] - 5s 5ms/step - l

In [ ]:
y_pred = ann.predict(X_test)

In [ ]:
y_pred = (y_pred>0.5)

In [ ]:
y_pred

array([[ True],
       [ True],
       [ True],
       ...,
       [False],
       [ True],
       [False]])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[5588,  288],
       [ 463, 4886]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      5876
           1       0.94      0.91      0.93      5349

    accuracy                           0.93     11225
   macro avg       0.93      0.93      0.93     11225
weighted avg       0.93      0.93      0.93     11225

